# prepare env

In [ ]:
!pip install OpenNMT-py==2.3.0

## or build from source to use command

In [1]:
!wget https://github.com/OpenNMT/OpenNMT-py/archive/refs/tags/2.3.0.tar.gz
!tar -zxvf 2.3.0.tar.gz
!mv OpenNMT-py-2.3.0 OpenNMT-py

--2025-02-05 10:10:44--  https://github.com/OpenNMT/OpenNMT-py/archive/refs/tags/2.3.0.tar.gz
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/OpenNMT/OpenNMT-py/tar.gz/refs/tags/2.3.0 [following]
--2025-02-05 10:10:45--  https://codeload.github.com/OpenNMT/OpenNMT-py/tar.gz/refs/tags/2.3.0
Resolving codeload.github.com (codeload.github.com)... 20.205.243.165
Connecting to codeload.github.com (codeload.github.com)|20.205.243.165|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/x-gzip]
Saving to: ‘2.3.0.tar.gz’

2.3.0.tar.gz            [         <=>        ]  77.81M  16.9MB/s    in 5.1s    

2025-02-05 10:10:51 (15.3 MB/s) - ‘2.3.0.tar.gz’ saved [81586137]

OpenNMT-py-2.3.0/
OpenNMT-py-2.3.0/.github/
OpenNMT-py-2.3.0/.github/workflows/
OpenNMT-py-2.3.0/.github/workflows/push.ym

# Notice (to allow resume pretrain)

to enable training from pretrain<br>
in OpenNMT-py/onmt/models/model_saver.py change (to bypass security safe)

```
def load_checkpoint(ckpt_path):

    import torch
    from onmt.inputters.text_dataset import TextMultiField

    # Add TextMultiField to the allowed safe globals
    torch.serialization.add_safe_globals([TextMultiField])
    """Load checkpoint from `ckpt_path` if any else return `None`."""
    checkpoint = None
    if ckpt_path:
        logger.info('Loading checkpoint from %s' % ckpt_path)
        checkpoint = torch.load(ckpt_path,
                                map_location=lambda storage, loc: storage, weights_only=False)
    return checkpoint
```

-------------------------------------------------------------------------

to enable translate from checkpoint <br>
in OpenNMT-py/onmt/model_builder.py add in line 81 (to bypass security safe)

```
def load_test_model(opt, model_path=None):
    import torch
    from onmt.inputters.text_dataset import TextMultiField

    # Add TextMultiField to the allowed safe globals
    torch.serialization.add_safe_globals([TextMultiField])

    if model_path is None:
        model_path = opt.models[0]
    checkpoint = torch.load(model_path,
                            map_location=lambda storage, loc: storage, weights_only=False)
```

In [2]:
# dont need to do this after above , can do before above 
%cd OpenNMT-py
!pip install -e .

[Errno 2] No such file or directory: 'OpenNMT-py'
/home/manh264/code_linux/manhMNT/src


/home/manh264/code_linux/hungNMT/env_4/lib/python3.10/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


Obtaining file:///home/manh264/code_linux/manhMNT/src
ERROR: file:///home/manh264/code_linux/manhMNT/src does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


# build vocab + training (no BPE)

In [5]:
cd /home/manh264/code_linux/manhMNT/src

/home/manh264/code_linux/manhMNT/src


In [6]:
config = """# data-no-bpe.yaml

## Where the samples will be written
save_data: data/run2/example
## Where the vocab(s) will be written
src_vocab: data/run2/example.vocab.src
tgt_vocab: data/run2/example.vocab.tgt
# Prevent overwriting existing files in the folder
overwrite: True

# Corpus opts:
data:
    corpus_1:
        path_src: data/src-train-token.txt
        path_tgt: data/tgt-train-token.txt
    valid:
        path_src: data/src-val-token.txt
        path_tgt: data/tgt-val-token.txt

# Vocabulary files that were just created
src_vocab: models/run2/example.vocab.src
tgt_vocab: models/run2/example.vocab.tgt

# Train on a single GPU
world_size: 1
gpu_ranks: [0]

# Where to save the checkpoints
save_model: models/run2/model
save_checkpoint_steps: 1000
train_steps: 60000
valid_steps: 1000

# Model
position_encoding: 'true'
enc_layers: 6
dec_layers: 6
decoder_type: transformer
encoder_type: transformer
word_vec_size: 512
rnn_size: 512
layers: 6
transformer_ff: 2048
heads: 8

# Batching
queue_size: 10000
batch_size: 4096
valid_batch_size: 4096
batch_type: tokens
"""

with open("khmer-viet-no-bpe.yaml", "w") as f:
    f.write(config)

build vocab and train

In [7]:
!onmt_build_vocab -config khmer-viet-no-bpe.yaml -n_sample 10000

/home/manh264/code_linux/hungNMT/src/OpenNMT-py/onmt/modules/sparse_activations.py:48: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(ctx, input, dim=0):
/home/manh264/code_linux/hungNMT/src/OpenNMT-py/onmt/modules/sparse_activations.py:68: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, grad_output):
/home/manh264/code_linux/hungNMT/src/OpenNMT-py/onmt/modules/sparse_losses.py:13: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(ctx, input, target):
/home/manh264/code_linux/hungNMT/src/OpenNMT-py/onmt/modules/sparse_losses.py:37: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  d

In [8]:
!onmt_train -config khmer-viet-no-bpe.yaml | tee output_log/train2.log

tee: output_log/train2.log: No such file or directory
/home/manh264/code_linux/hungNMT/src/OpenNMT-py/onmt/modules/sparse_activations.py:48: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(ctx, input, dim=0):
/home/manh264/code_linux/hungNMT/src/OpenNMT-py/onmt/modules/sparse_activations.py:68: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, grad_output):
/home/manh264/code_linux/hungNMT/src/OpenNMT-py/onmt/modules/sparse_losses.py:13: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(ctx, input, target):
/home/manh264/code_linux/hungNMT/src/OpenNMT-py/onmt/modules/sparse_losses.py:37: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp

In [1]:
# Define a Python variable for the source file path
model_path = "models/run2/model_step_30000.pt"
src = "data/fake-src-test-token.txt"
output_path = "data/pred_no_bpe.txt"
log_output_path = "data/translate.log"
# Use the variable in the shell command (note the curly braces around src)
!onmt_translate -model {model_path} -src {src} -output {output_path} -verbose -gpu 0 | tee {log_output_path}


/home/manh264/code_linux/hungNMT/src/OpenNMT-py/onmt/modules/sparse_activations.py:48: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(ctx, input, dim=0):
/home/manh264/code_linux/hungNMT/src/OpenNMT-py/onmt/modules/sparse_activations.py:68: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, grad_output):
/home/manh264/code_linux/hungNMT/src/OpenNMT-py/onmt/modules/sparse_losses.py:13: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(ctx, input, target):
/home/manh264/code_linux/hungNMT/src/OpenNMT-py/onmt/modules/sparse_losses.py:37: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  d

# build vocab + training (have BPE)

In [3]:
config = """# khmer-viet.yaml

## Where the samples will be written
save_data: data/run/example
## Where the vocab(s) will be written
src_vocab: data/run/example.vocab.src
tgt_vocab: data/run/example.vocab.tgt
# Prevent overwriting existing files in the folder
overwrite: True

# Corpus opts:
data:
    corpus_1:
        path_src: data/src-train-bpe.txt
        path_tgt: data/tgt-train-bpe.txt
    valid:
        path_src: data/src-val-bpe.txt
        path_tgt: data/tgt-val-bpe.txt

# Vocabulary files that were just created
src_vocab: models/run/example.vocab.src
tgt_vocab: models/run/example.vocab.tgt

# Train on a single GPU
world_size: 1
gpu_ranks: [0]

# Where to save the checkpoints
save_model: models/run/model
save_checkpoint_steps: 1000
train_steps: 30000
valid_steps: 1000

# Model
position_encoding: 'true'
enc_layers: 6
dec_layers: 6
decoder_type: transformer
encoder_type: transformer
word_vec_size: 512
rnn_size: 512
layers: 6
transformer_ff: 2048
heads: 8

# Batching
queue_size: 10000
batch_size: 4096
valid_batch_size: 4096
batch_type: tokens
"""
with open("khmer-viet.yaml", "w") as f:
    f.write(config)

## BPE encoding 

In [ ]:
!python OpenNMT-py/tools/learn_bpe.py -i ./data/src-train-token.txt -o ./data/src.code -s 10000

In [ ]:
!python OpenNMT-py/tools/learn_bpe.py -i ./data/tgt-train-token.txt -o ./data/tgt.code -s 10000

In [ ]:
!python OpenNMT-py/tools/apply_bpe.py -c ./data/src.code -i ./data/src-train-token.txt -o ./data/src-train-bpe.txt

In [ ]:
!python OpenNMT-py/tools/apply_bpe.py -c ./data/src.code -i ./data/src-val-token.txt -o ./data/src-val-bpe.txt

In [ ]:
!python OpenNMT-py/tools/apply_bpe.py -c ./data/tgt.code -i ./data/src-test-token.txt -o OpenNMT-py/data/src-test-bpe.txt

In [ ]:
!python OpenNMT-py/tools/apply_bpe.py -c ./data/tgt.code -i ./data/tgt-train-token.txt -o ./data/tgt-train-bpe.txt

In [ ]:
!python OpenNMT-py/tools/apply_bpe.py -c ./data/tgt.code -i ./data/tgt-val-token.txt -o ./data/tgt-val-bpe.txt

create vocab + TRAIN

In [ ]:
!onmt_build_vocab -config khmer-viet.yaml -n_sample 10000

In [ ]:
!onmt_train -config khmer-viet.yaml --verbose

In [ ]:
!onmt_translate -model models/run/model_step_5000.pt -src data/src-train-token.txt -output data/pred_bpe.txt -verbose -gpu 0